<a href="https://colab.research.google.com/github/khushdeep95/Flood-sense/blob/main/Flood_Sense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
BASE_PATH = "/content/drive/MyDrive/FloodSense/"

In [3]:
import os

base_path = "/content/drive/MyDrive/FloodSense"

os.makedirs(base_path, exist_ok=True)
os.makedirs(base_path + "/data", exist_ok=True)
os.makedirs(base_path + "/output", exist_ok=True)

print("Project folders created!")

Project folders created!


In [4]:
!pip install geopandas osmnx folium requests scikit-learn shapely openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 3.3 MB/s eta 0:00:00


In [5]:
from google.colab import files
files.upload()

Saving kaggle (4).json to kaggle (4).json


{'kaggle (4).json': b'{"username":"khushdeep058","key":"05a665872c51f72b3a6e2db03451fb98"}'}

In [6]:
!mv "kaggle (4).json" kaggle.json

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d wigglerofgems/open-city-delhi-wards-shape-dataset-kml

Dataset URL: https://www.kaggle.com/datasets/wigglerofgems/open-city-delhi-wards-shape-dataset-kml
License(s): ODbL-1.0
  0% 0.00/1.84M [00:00<?, ?B/s]
100% 1.84M/1.84M [00:00<00:00, 703MB/s]


In [9]:
!unzip open-city-delhi-wards-shape-dataset-kml.zip -d delhi_wards

Archive:  open-city-delhi-wards-shape-dataset-kml.zip
  inflating: delhi_wards/delhi_wards.kml  


In [10]:
!pip install geopandas shapely fiona pyproj

In [11]:
!pip install geopandas fiona shapely pyproj --upgrade

In [12]:
!apt-get install -y gdal-bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-gdal python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-dev python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 2 not upgraded.
Need to get 5,168 kB of archives.
After this operation, 25.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.8.4+dfsg-1~jammy0 [1,095 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.8.4+dfsg-1~jammy0 [605 kB]
Fetched 5,168 kB in 3s (1,485 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 117540 file

In [13]:
!ogr2ogr -f "GeoJSON" delhi_wards/wards.geojson delhi_wards/delhi_wards.kml

In [14]:
import geopandas as gpd

wards = gpd.read_file("delhi_wards/wards.geojson")

wards.head()

,tessellate,extrude,visibility,FID,WNo_SEC,AC_No,AC_No_1,AC_Name,Ward_No,WardName,TotalPop,SC_Pop,NW2022,geometry
0,-1,0,-1,0,100,28,28,HARI NAGAR,100,FATEH NAGAR,57335,2034,"100, FATEH NAGAR","POLYGON ((77.10332 28.63439, 77.10335 28.63419..."
1,-1,0,-1,1,113,32,32,UTTAM NAGAR,113,MOHAN GARDEN,60029,5854,"113, MOHAN GARDEN","POLYGON ((77.03531 28.62545, 77.03528 28.62543..."
2,-1,0,-1,2,115,32,32,UTTAM NAGAR,115,UTTAM NAGAR,68309,3410,"115, UTTAM NAGAR","POLYGON ((77.05277 28.62099, 77.05277 28.62077..."
3,-1,0,-1,3,116,32,32,UTTAM NAGAR,116,BINDA PUR,58750,6992,"116, BINDA PUR","POLYGON ((77.0606 28.6131, 77.0606 28.6131, 77..."
4,-1,0,-1,4,12,3,3,TIMARPUR,12,MALKA GANJ,68941,22081,"12, MALKA GANJ","POLYGON ((77.21049 28.67505, 77.21055 28.67503..."


In [15]:
wards = wards.to_crs(epsg=4326)  # Ensure lat/lon

In [16]:
# Step 1: Convert to projected CRS
wards_projected = wards.to_crs(epsg=3857)

# Step 2: Calculate centroid in meters
wards_projected["centroid"] = wards_projected.geometry.centroid

# Step 3: Convert centroid back to lat/lon
wards["lat"] = wards_projected["centroid"].to_crs(epsg=4326).y
wards["lon"] = wards_projected["centroid"].to_crs(epsg=4326).x

In [17]:
wards[["Ward_No", "WardName", "lat", "lon"]].head()

,Ward_No,WardName,lat,lon
0,100,FATEH NAGAR,28.627466,77.098704
1,113,MOHAN GARDEN,28.626414,77.029077
2,115,UTTAM NAGAR,28.616959,77.051736
3,116,BINDA PUR,28.611384,77.068221
4,12,MALKA GANJ,28.683715,77.209969


In [18]:
import requests

API_KEY = "1fb2b126268e3ef2e82c7cb3e207da6e"

def get_rainfall_24h(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"

    response = requests.get(url)
    data = response.json()

    total_rain = 0

    for item in data["list"][:8]:
        if "rain" in item and "3h" in item["rain"]:
            total_rain += item["rain"]["3h"]

    return total_rain

In [19]:
wards["rainfall_mm"] = wards.apply(
    lambda row: get_rainfall_24h(row["lat"], row["lon"]),
    axis=1
)

In [20]:
lat = wards.iloc[0]["lat"]
lon = wards.iloc[0]["lon"]

get_rainfall_24h(lat, lon)

0

In [21]:
wards[["WardName", "rainfall_mm"]].head()

,WardName,rainfall_mm
0,FATEH NAGAR,0
1,MOHAN GARDEN,0
2,UTTAM NAGAR,0
3,BINDA PUR,0
4,MALKA GANJ,0


In [45]:
def apply_rain_scenario(mode, real_rain):

    if mode == "normal":
        rainfall_value = real_rain
        print("🟢 Normal Forecast Mode")

    elif mode == "heavy":
        rainfall_value = 60
        print("🟡 Heavy Rain Simulation")

    elif mode == "extreme":
        rainfall_value = 120
        print("🔴 Extreme Monsoon Burst Simulation")

    else:
        raise ValueError("Invalid rain mode selected")

    wards["rainfall_mm"] = rainfall_value
    print(f"Rainfall Applied: {rainfall_value} mm")
    return rainfall_value


# Run Demo
apply_rain_scenario("heavy", real_rain)

NameError: name 'real_rain' is not defined

In [22]:
import requests

def get_elevation(lat, lon):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    response = requests.get(url)
    data = response.json()
    return data["results"][0]["elevation"]

In [23]:
get_elevation(wards.iloc[0]["lat"], wards.iloc[0]["lon"])

218.0

In [24]:
wards["elevation_m"] = wards.apply(
    lambda row: get_elevation(row["lat"], row["lon"]),
    axis=1
)

In [25]:
wards[["WardName", "elevation_m"]].head()

,WardName,elevation_m
0,FATEH NAGAR,218.0
1,MOHAN GARDEN,213.0
2,UTTAM NAGAR,217.0
3,BINDA PUR,216.0
4,MALKA GANJ,218.0


In [26]:
ELEVATION_DEMO = False   # change to True for demo simulation

if ELEVATION_DEMO:
    print("🔴 Demo Mode: Simulating Low-Lying Flood Zones")
    wards["elevation_m"] = 8   # simulate low elevation
else:
    print("🟢 Using Real Elevation Data")

🟢 Using Real Elevation Data


In [27]:
import osmnx as ox

# Merge all wards into one Delhi boundary
delhi_boundary = wards.geometry.union_all()

# Fetch waterways once
tags = {"waterway": True}
all_waterways = ox.features_from_polygon(delhi_boundary, tags)

print("Waterways fetched:", len(all_waterways))

Waterways fetched: 952


In [28]:
# Project wards to meters
wards_projected = wards.to_crs(epsg=3857)

# Project waterways to meters
all_waterways = all_waterways.to_crs(epsg=3857)

In [29]:
wards_projected["drainage_density"] = 0.0
for idx, row in wards_projected.iterrows():

    ward_geom = row.geometry

    # Find waterways intersecting this ward
    intersecting = all_waterways[all_waterways.intersects(ward_geom)]

    if not intersecting.empty:
        total_length = intersecting.length.sum()   # meters
        area = ward_geom.area                     # square meters

        density = total_length / area

        wards_projected.at[idx, "drainage_density"] = density

In [30]:
wards["drainage_density"] = wards_projected["drainage_density"]
wards[["WardName", "drainage_density"]].head()

,WardName,drainage_density
0,FATEH NAGAR,0.000000
1,MOHAN GARDEN,0.020553
2,UTTAM NAGAR,0.000000
3,BINDA PUR,0.000000
4,MALKA GANJ,0.002199


In [51]:
tags = {
    "landuse": ["residential", "commercial", "industrial"]
}

all_landuse = ox.features_from_polygon(delhi_boundary, tags)

print("Built-up features fetched:", len(all_landuse))

Built-up features fetched: 3498


In [52]:
all_landuse = all_landuse.to_crs(epsg=3857)
wards_projected = wards.to_crs(epsg=3857)
all_landuse = all_landuse[all_landuse.geometry.type.isin(["Polygon", "MultiPolygon"])]
wards_projected["impermeable_ratio"] = 0.0
for idx, row in wards_projected.iterrows():

    ward_geom = row.geometry

    intersecting = all_landuse[all_landuse.intersects(ward_geom)]

    if not intersecting.empty:

        # Clip built-up areas to ward
        clipped = intersecting.clip(ward_geom)

        built_area = clipped.area.sum()   # m²
        ward_area = ward_geom.area        # m²

        ratio = built_area / ward_area

        wards_projected.at[idx, "impermeable_ratio"] = ratio

In [53]:
wards["impermeable_ratio"] = wards_projected["impermeable_ratio"]
wards[["WardName", "impermeable_ratio"]].head()

,WardName,impermeable_ratio
0,FATEH NAGAR,0.451759
1,MOHAN GARDEN,0.652234
2,UTTAM NAGAR,0.364855
3,BINDA PUR,0.361199
4,MALKA GANJ,0.133879


In [54]:
import pandas as pd
import numpy as np

training_data = []

rain_scenarios = [10, 30, 60, 90, 120]

for rain in rain_scenarios:
    temp = wards.copy()
    temp["rainfall_mm"] = rain

    # Domain-based risk logic (for training only)
    temp["risk_score"] = (
        0.7 * (temp["rainfall_mm"] / 120) +
        0.15 * (1 - temp["elevation_m"] / temp["elevation_m"].max()) +
        0.10 * temp["impermeable_ratio"] +
        0.05 * (1 - temp["drainage_density"] / temp["drainage_density"].max())
    )

    temp["flood_label"] = (temp["risk_score"] > 0.6).astype(int)

    training_data.append(temp)

train_df = pd.concat(training_data, ignore_index=True)

In [55]:
from sklearn.ensemble import RandomForestClassifier

features = ["rainfall_mm", "elevation_m", "drainage_density", "impermeable_ratio"]

X_train = train_df[features]
y_train = train_df["flood_label"]

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=300, random_state=42)

In [58]:
def run_simulation(mode):

    apply_rain_scenario(mode, real_rain)

    X_demo = wards[["rainfall_mm", "elevation_m", "drainage_density", "impermeable_ratio"]]

    wards["flood_probability"] = model.predict_proba(X_demo)[:, 1]

    m = folium.Map(location=[28.6, 77.2], zoom_start=11)

    def get_color(prob):
        if prob > 0.75:
            return "darkred"
        elif prob > 0.6:
            return "red"
        elif prob > 0.4:
            return "orange"
        else:
            return "green"

    for _, row in wards.iterrows():
        folium.GeoJson(
            row.geometry,
            style_function=lambda x, prob=row["flood_probability"]: {
                "fillColor": get_color(prob),
                "color": "black",
                "weight": 1,
                "fillOpacity": 0.6,
            },
            tooltip=f"{row['WardName']} | Flood Probability: {row['flood_probability']:.2f}"
        ).add_to(m)

    return m